# Introduction


## Challenge Large Scale Machine Learning

### Authors: 
#### Pavlo Mozharovskyi (pavlo.mozharovskyi@telecom-paristech.fr), Umut Şimşekli (umut.simsekli@telecom-paristech.fr)


### Fusion of algorithms for face recognition

The increasingly ubiquitous presence of biometric solutions and face recognition in particular in everyday life requires their adaptation for practical scenario. In the presence of several possible solutions, and if global decisions are to be made, each such single solution can be far less efficient than tailoring them to the complexity of an image.

In this challenge, the goal is to build a fusion of algorithms in order to construct the best suited solution for comparison of a pair of images. This fusion will be driven by qualities computed on each image.

Comparing of two images is done in two steps. 1st, a vector of features is computed for each image. 2nd, a simple function produces a vector of scores for a pair of images. The goal is to create a function that will compare a pair of images based on the information mentioned above, and decide whether two images belong to the same person.

You are provided a label set of training data and a test set without labels. You should submit a .csv file with labels for each entry of this test set.

# The properties of the dataset:


### Training data: 


The training set consist of two files, **xtrain_challenge.csv** and **xtest_challenge.csv**.

File **xtrain_challenge.csv** contains one observation per row which contains following entries based on a pair of images, A and B say:
 * columns 1-14 - 14 qualities on image A;
 * columns 15-28 - 14 qualities on image B;
 * columns 29-36 - 8 matching scores between A and B.

File **ytrain_challenge.csv** contains one line with each entry corresponding to one observation in **xtrain_challenge.csv**, maintaining the order, and has '1' if a pair of images belong to the same person and '0' otherwise.

There are in total 3.196.465 training observations.

### Test data:

File **xtest_challenge.csv** has the same structure as file **xtrain_challenge.csv**.

There are in total 1.598.219 test observations.

## The performance criterion¶

The performance criterion is the **prediction accuracy** on the test set, which is a value between 0 and 1, the higher the better.

# Training Data

Training data, input (file **xtrain_challenge.csv**): https://www.dropbox.com/s/myvvtmw61eg5gk7/xtrain_challenge.csv

Training data, output (file **ytrain_challenge.csv**): https://www.dropbox.com/s/cleumxob0dfzre4/ytrain_challenge.csv

# Test Data 

Test data, input (file **xtest_challenge.csv**): https://www.dropbox.com/s/bfrx8b4mqythm4q/xtest_challenge.csv

# Import package

In [2]:
%matplotlib inline
import numpy as np
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import math

## Load and investigate the data

## Decouverte du dataset

In [3]:
xtrain = pd.read_csv('xtrain_challenge.csv')
ytrain = pd.read_csv('ytrain_challenge.csv')
dataset=xtrain.merge(ytrain,  left_index=True,right_index=True)

In [3]:
dataset.describe()

,fA1,fA2,fA3,fA4,fA5,fA6,fA7,fA8,fA9,fA10,...,fB14,s1,s2,s3,s4,s5,s6,s7,s8,y
count,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,...,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06
mean,3.123656e+00,7.200140e-01,8.895844e-02,2.938227e+01,9.112188e-01,4.165448e-03,8.456615e-02,1.323673e-01,2.970159e-02,-2.833317e-04,...,2.477718e+02,3.874350e+03,2.434998e+03,4.200151e+03,4.242067e+03,3.488791e+03,3.399066e+03,3.509614e+03,3.745971e+03,2.857347e-01
std,1.618458e+00,4.224322e-01,2.824698e-01,9.793075e+00,2.757797e-01,4.433567e-02,2.729676e-01,4.159110e-01,1.279389e-01,2.096133e-02,...,1.221232e+02,2.063822e+03,2.272729e+02,2.954367e+03,2.127659e+03,1.360081e+03,1.363354e+03,1.066627e+03,1.548290e+03,4.517637e-01
min,-4.500000e-01,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.550000e+00,-3.030000e+00,-3.000000e-01,...,-3.087900e+02,1.212700e+03,1.511400e+03,7.218000e+02,1.133800e+03,6.682000e+02,7.200000e+02,1.358000e+03,7.107000e+02,0.000000e+00
25%,1.950000e+00,2.500000e-01,0.000000e+00,2.200000e+01,1.000000e+00,0.000000e+00,0.000000e+00,-2.000000e-02,-2.000000e-02,0.000000e+00,...,2.214200e+02,2.420300e+03,2.296300e+03,2.086200e+03,2.735400e+03,2.457200e+03,2.335000e+03,2.742600e+03,2.620100e+03,0.000000e+00
50%,3.390000e+00,1.000000e+00,0.000000e+00,2.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,...,2.766400e+02,3.062600e+03,2.439800e+03,3.073300e+03,3.402000e+03,3.063300e+03,2.999800e+03,3.151800e+03,3.218800e+03,0.000000e+00
75%,4.380000e+00,1.000000e+00,0.000000e+00,3.400000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e-01,8.000000e-02,1.000000e-02,...,3.216600e+02,4.442500e+03,2.589300e+03,5.143800e+03,4.898800e+03,4.406800e+03,4.375500e+03,3.926300e+03,4.396900e+03,1.000000e+00
max,7.940000e+00,1.000000e+00,1.000000e+00,7.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,2.500000e+00,7.600000e-01,2.600000e-01,...,4.814200e+02,1.204420e+04,3.035900e+03,1.666650e+04,1.181290e+04,7.731000e+03,7.580100e+03,6.949500e+03,8.524900e+03,1.000000e+00


In [4]:
print(dataset.columns)
print(dataset.dtypes)

Index(['fA1', 'fA2', 'fA3', 'fA4', 'fA5', 'fA6', 'fA7', 'fA8', 'fA9', 'fA10',
       'fA11', 'fA12', 'fA13', 'fA14', 'fB1', 'fB2', 'fB3', 'fB4', 'fB5',
       'fB6', 'fB7', 'fB8', 'fB9', 'fB10', 'fB11', 'fB12', 'fB13', 'fB14',
       's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 'y'],
      dtype='object')
fA1     float64
fA2     float64
fA3     float64
fA4       int64
fA5     float64
fA6     float64
fA7     float64
fA8     float64
fA9     float64
fA10    float64
fA11    float64
fA12    float64
fA13    float64
fA14    float64
fB1     float64
fB2     float64
fB3     float64
fB4       int64
fB5     float64
fB6     float64
fB7     float64
fB8     float64
fB9     float64
fB10    float64
fB11    float64
fB12    float64
fB13    float64
fB14    float64
s1      float64
s2      float64
s3      float64
s4      float64
s5      float64
s6      float64
s7      float64
s8      float64
y         int64
dtype: object


In [5]:
dataset.corr()[['s1','s2','s3','s4','s5','s6','s7','s8']]

,s1,s2,s3,s4,s5,s6,s7,s8
fA1,0.302163,0.175194,0.175699,0.319515,0.113217,0.072562,0.221904,0.328958
fA2,0.195161,0.094589,0.108425,0.199246,0.085430,0.065890,0.127893,0.207402
fA3,-0.156674,-0.077087,-0.057989,-0.170331,-0.014477,0.014819,-0.092125,-0.190083
fA4,-0.221524,-0.171908,-0.180561,-0.221287,-0.170143,-0.141654,-0.210691,-0.223554
fA5,0.267400,0.005037,0.011938,0.299303,-0.108870,-0.170258,0.090483,0.339884
fA6,-0.031351,-0.070376,-0.037729,-0.029526,-0.030271,-0.032501,-0.038793,-0.028510
fA7,-0.265017,0.006394,-0.005890,-0.297546,0.114937,0.177315,-0.085074,-0.338711
fA8,-0.217639,0.007184,0.018990,-0.247403,0.132473,0.180096,-0.053877,-0.284671
fA9,0.053327,0.074944,0.032096,0.055311,0.025940,0.026823,0.047415,0.054904
fA10,0.005469,0.025288,0.002957,0.006599,-0.001799,-0.003317,0.004837,0.008699


In [ ]:
dataset.corr()['y']

fA1     0.172994
fA2     0.111962
fA3    -0.032684
fA4    -0.206904
fA5     0.013986
fA6    -0.000138
fA7    -0.014096
fA8     0.001426
fA9     0.050191
fA10   -0.011532
fA11   -0.127774
fA12    0.129836
fA13   -0.043042
fA14    0.093309
fB1     0.149043
fB2     0.114148
fB3    -0.022911
fB4    -0.190803
fB5     0.012304
fB6     0.047056
fB7    -0.074319
fB8    -0.055535
fB9     0.081903
fB10   -0.010141
fB11   -0.127597
fB12    0.136991
fB13   -0.030770
fB14    0.071156
s1      0.801645
s2      0.325149
s3      0.771318
s4      0.804455
s5      0.761075
s6      0.771061
s7      0.809114
s8      0.795365
y       1.000000
Name: y, dtype: float64

On observe que le matching ( colonne s1 à s8) est fortement corrèlé avec le matching des variables.

In [ ]:
dataset['y'].value_counts()

0    2283124
1     913341
Name: y, dtype: int64

On observe que la distribution des variables à prédire n'est pas également répartie.

## split des données

On a vu précédemment que la classe à prédire n'est pas également répartie il faut donc en tenir compte lors du split pour ne pas accroittre cette disparité. Le parametre `stratify` est prévu à cet effet.  
Dans le cas d'utilisation de `réseau de neuronnes` et de `light GBM`, Centrer-réduire les données à un réel impact  sur les perfmances des modèles. Cette étape a été réalisée plus bas.

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(dataset,test_size=0.1,stratify=dataset['y'])
data=pd.read_csv('dataNormalized.csv',index_col=0)
trainN,testN=train_test_split(data,test_size=0.1,stratify=data['y'])

## Modelling

### Choix des modèles

Suite à une courte recherche bibliographique j'ai décidé de me tourner vers 5 modèles différents:   
 * LDA
 * SVM
 * Réseaux de neuronnes  
 * XGBoost
 * LightGBoost
 
La `SVM` et la `LDA` n'ont pas offert assez bon résultat face aux autres mdèles pour justifier de détailler leur implémentation.

La suite de mon rapport rendra compte de l'implémentation d'un réseau de neuronnes et des modèles de gradient boosting

### Puissance de calcul pour l'optimisation des hyper-parametres

Afin d'optimiser le temps de calcul pour trouver les hyperparametres des trois modèles restants, les calculs ont été distribué sur deux clusters dask composé des machines de l'école. Un notebook pour chaque modèle a aussi été mis en place pour pouvoir lancer les calculs simultanéments.

Voici le lien vers chacun de ces notebooks:  
 * Neural network
 * XGBoost
 * LGBoost
 
 

### Resultats du LGBoost

In [ ]:
pd.read_csv('LGBM.csv')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_num_leaves,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,98.779547,0.946638,14.711493,0.277883,0.02,256,"{'learning_rate': 0.02, 'num_leaves': 256}",0.987912,0.987832,0.987872,0.000040,1,0.990968,0.991069,0.991019,0.000050
1,94.529286,0.926894,13.868437,0.247973,0.02,128,"{'learning_rate': 0.02, 'num_leaves': 128}",0.986516,0.986479,0.986498,0.000018,2,0.988014,0.988184,0.988099,0.000085
2,68.146726,0.511424,11.636487,0.263768,0.02,64,"{'learning_rate': 0.02, 'num_leaves': 64}",0.985063,0.984973,0.985018,0.000045,3,0.985731,0.985950,0.985840,0.000110
3,58.402427,4.384872,9.723119,0.349699,0.02,32,"{'learning_rate': 0.02, 'num_leaves': 32}",0.983464,0.983290,0.983377,0.000087,4,0.983764,0.983920,0.983842,0.000078
4,28.826629,0.630166,7.994866,0.201128,0.40,32,"{'learning_rate': 0.4, 'num_leaves': 32}",0.983425,0.983252,0.983339,0.000087,5,0.984965,0.984797,0.984881,0.000084
5,26.649774,0.406546,8.610677,0.083260,0.70,32,"{'learning_rate': 0.7, 'num_leaves': 32}",0.981956,0.981920,0.981938,0.000018,6,0.983401,0.983681,0.983541,0.000140
6,35.504586,0.599904,10.101779,0.123615,0.40,64,"{'learning_rate': 0.4, 'num_leaves': 64}",0.980784,0.981377,0.981080,0.000296,7,0.982666,0.983824,0.983245,0.000579
7,34.976297,0.386149,10.287855,0.128522,0.70,64,"{'learning_rate': 0.7, 'num_leaves': 64}",0.979862,0.980125,0.979993,0.000131,8,0.981306,0.981954,0.981630,0.000324
8,50.079604,1.605808,12.735095,0.204151,0.40,128,"{'learning_rate': 0.4, 'num_leaves': 128}",0.976314,0.979722,0.978018,0.001704,9,0.979315,0.982140,0.980728,0.001412
9,70.000215,1.332182,16.317688,0.151828,0.40,256,"{'learning_rate': 0.4, 'num_leaves': 256}",0.968426,0.970547,0.969486,0.001061,10,0.971408,0.973724,0.972566,0.001158


In [ ]:
modelLGBM = joblib.load('LGBM_final.sav')
yvalid = modelLGBM.predict(testN.drop('y',axis=1))
(yvalid == testN['y']).mean()

### Resultat du XGBoost

In [ ]:
pd.read_csv('XGB.csv')

In [ ]:
modelXGB = joblib.load('XGB_final.sav')
yvalid = modelXGB.predict(test.drop('y',axis=1))
(yvalid == test['y']).mean()

### Resultat du réseau de neurones

In [ ]:
pd.read_csv('NN.csv')

In [ ]:
modelNN = joblib.load('NN_final.sav')
yvalid = modelNN.predict(testN.drop('y',axis=1))
(yvalid == testN['y']).mean()

## Prepare a file for submission

Le modèle avec le meilleur score est le modèle `LightGBM`.

In [9]:
# Load test data
xtest = pd.read_csv('testNormalized.csv')
# Classify the provided test data
ytest = modelLGBM.predict(xtest)
print(ytest.shape)
np.savetxt('ytest_final.csv', ytest, fmt = '%1.0d', delimiter=',')

(1598219,)


#### Now it's your turn. Good luck !  :) 